In [5]:
%load_ext autoreload
%autoreload 2



%autoreload 2
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from scipy.spatial.distance import pdist, squareform
from scipy.special import expit
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering  # Add this import

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Comparie pis

In [15]:
pi_batch = torch.load("/Users/sarahurbut/Library/CloudStorage/Dropbox/pi_enroll_age_offset_0_sex_0_10000.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_49933/1726899128.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_batch = torch.load("/Users/sarahurbut/Libr

In [16]:
pi_original=torch.load("/Users/sarahurbut/Library/CloudStorage/Dropbox/pi_enroll_sex_0_10000.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_49933/3065523748.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_original=torch.load("/Users/sarahurbut/Lib

In [17]:
are_equal = torch.allclose(pi_original, pi_batch, atol=1e-6)
print("Are the interactive and batch predictions the same?", are_equal)

Are the interactive and batch predictions the same? True


# fo the training pis


In [9]:
pi_batch = torch.load("/Users/sarahurbut/Library/CloudStorage/Dropbox/pi_enroll_age_offset_0_sex_20000_30000.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_49933/3860175492.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_batch = torch.load("/Users/sarahurbut/Libr

In [10]:
pi_original=torch.load("/Users/sarahurbut/Library/CloudStorage/Dropbox/pi_enroll_sex_20000_30000.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_49933/1471984263.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pi_original=torch.load("/Users/sarahurbut/Lib

In [11]:
are_equal = torch.allclose(pi_original, pi_batch, atol=1e-4)
print("Are the interactive and batch predictions the same?", are_equal)

Are the interactive and batch predictions the same? True


### arrange pis

In [13]:
import pandas as pd
fh_processed=pd.read_csv('/Users/sarahurbut/Library/Cloudstorage/Dropbox/baselinagefamh.csv')
len(fh_processed)



pce_df_subset = fh_processed.iloc[0:10000].reset_index(drop=True)
sex=pce_df_subset['sex'].values


In [18]:
import torch
import numpy as np

# Parameters
years_to_use = 10
N = len(pce_df_subset)  # Number of people
# Load one batch to get D (number of diseases) and T (number of time points)
sample_pi = torch.load("/Users/sarahurbut/Library/CloudStorage/Dropbox/pi_enroll_age_offset_0_sex_0_10000.pt")
D = sample_pi.shape[1]
T = sample_pi.shape[2]

# Load all batch predictions into a list
pi_batches = [
    torch.load(f"/Users/sarahurbut/Library/CloudStorage/Dropbox/pi_enroll_age_offset_{k}_sex_0_10000.pt")
    for k in range(years_to_use)
]

# Get enrollment ages
enrollment_ages = pce_df_subset['age'].to_numpy()

# Initialize output array
pi_for_cox = torch.zeros((N, D, years_to_use))

for i in range(N):
    t = int(enrollment_ages[i] - 30)
    for k in range(years_to_use):
        if t + k < T:
            pi_for_cox[i, :, k] = pi_batches[k][i, :, t + k]
        else:
            pi_for_cox[i, :, k] = float('nan')  # or 0, depending on your preference

# Save for R or further use
torch.save(pi_for_cox, "pi_for_cox.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_49933/3850383601.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sample_pi = torch.load("/Users/sarahurbut/Lib

In [19]:
pi_for_cox.shape

torch.Size([10000, 348, 10])

In [20]:
# Pick a few random indices to check
indices_to_check = [0, 1, 2]  # or use np.random.choice(N, 3, replace=False)

for idx in indices_to_check:
    t = int(enrollment_ages[idx] - 30)
    print(f"\nPerson {idx} (enrollment age: {enrollment_ages[idx]}, t={t}):")
    for k in range(years_to_use):
        # Value from assembled array
        val_from_cox = pi_for_cox[idx, 0, k].item()  # disease 0 as example
        # Value from batch file
        val_from_batch = pi_batches[k][idx, 0, t + k].item() if t + k < T else float('nan')
        print(f"  Year {k}: pi_for_cox={val_from_cox:.6f}, pi_batch={val_from_batch:.6f}, match={np.isclose(val_from_cox, val_from_batch, atol=1e-6)}")


Person 0 (enrollment age: 69, t=39):
  Year 0: pi_for_cox=0.000230, pi_batch=0.000230, match=True
  Year 1: pi_for_cox=0.000224, pi_batch=0.000224, match=True
  Year 2: pi_for_cox=0.000229, pi_batch=0.000229, match=True
  Year 3: pi_for_cox=0.000230, pi_batch=0.000230, match=True
  Year 4: pi_for_cox=0.000224, pi_batch=0.000224, match=True
  Year 5: pi_for_cox=0.000185, pi_batch=0.000185, match=True
  Year 6: pi_for_cox=0.000180, pi_batch=0.000180, match=True
  Year 7: pi_for_cox=0.000177, pi_batch=0.000177, match=True
  Year 8: pi_for_cox=0.000179, pi_batch=0.000179, match=True
  Year 9: pi_for_cox=0.000178, pi_batch=0.000178, match=True

Person 1 (enrollment age: 44, t=14):
  Year 0: pi_for_cox=0.000143, pi_batch=0.000143, match=True
  Year 1: pi_for_cox=0.000166, pi_batch=0.000166, match=True
  Year 2: pi_for_cox=0.000192, pi_batch=0.000192, match=True
  Year 3: pi_for_cox=0.000216, pi_batch=0.000216, match=True
  Year 4: pi_for_cox=0.000253, pi_batch=0.000253, match=True
  Year 5:

In [21]:
import torch
import numpy as np

years_to_use = 10
N = len(pce_df_subset)
D = pi_batches[0].shape[1]
T = pi_batches[0].shape[2]  # Full time axis

# Load all batch predictions into a list as before
pi_batches = [
    torch.load(f"/Users/sarahurbut/Library/CloudStorage/Dropbox/pi_enroll_age_offset_{k}_sex_0_10000.pt")
    for k in range(years_to_use)
]

enrollment_ages = pce_df_subset['age'].to_numpy()

# Initialize the full array with NaNs (or zeros)
pi_full = torch.full((N, D, T), float('nan'))

for i in range(N):
    t = int(enrollment_ages[i] - 30)
    for k in range(years_to_use):
        if t + k < T:
            pi_full[i, :, t + k] = pi_batches[k][i, :, t + k]
        # else: leave as NaN

# Save for R
torch.save(pi_full, "pi_full_leakage_free.pt")

/var/folders/fl/ng5crz0x0fnb6c6x8dk7tfth0000gn/T/ipykernel_49933/1761418030.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(f"/Users/sarahurbut/Library/Cloud

In [23]:
# Pick a few random indices to check
indices_to_check = [0, 1, 2]  # or use np.random.choice(N, 3, replace=False)

for idx in indices_to_check:
    t = int(enrollment_ages[idx] - 30)
    print(f"\nPerson {idx} (enrollment age: {enrollment_ages[idx]}, t={t}):")
    for k in range(years_to_use):
        # Value from assembled array
        val_from_cox = pi_full[idx, 0, t+k].item()  # disease 0 as example
        # Value from batch file
        val_from_batch = pi_batches[k][idx, 0, t + k].item() if t + k < T else float('nan')
        print(f"  Year {k}: pi_full={val_from_cox:.6f}, pi_batch={val_from_batch:.6f}, match={np.isclose(val_from_cox, val_from_batch, atol=1e-6)}")


Person 0 (enrollment age: 69, t=39):
  Year 0: pi_full=0.000230, pi_batch=0.000230, match=True
  Year 1: pi_full=0.000224, pi_batch=0.000224, match=True
  Year 2: pi_full=0.000229, pi_batch=0.000229, match=True
  Year 3: pi_full=0.000230, pi_batch=0.000230, match=True
  Year 4: pi_full=0.000224, pi_batch=0.000224, match=True
  Year 5: pi_full=0.000185, pi_batch=0.000185, match=True
  Year 6: pi_full=0.000180, pi_batch=0.000180, match=True
  Year 7: pi_full=0.000177, pi_batch=0.000177, match=True
  Year 8: pi_full=0.000179, pi_batch=0.000179, match=True
  Year 9: pi_full=0.000178, pi_batch=0.000178, match=True

Person 1 (enrollment age: 44, t=14):
  Year 0: pi_full=0.000143, pi_batch=0.000143, match=True
  Year 1: pi_full=0.000166, pi_batch=0.000166, match=True
  Year 2: pi_full=0.000192, pi_batch=0.000192, match=True
  Year 3: pi_full=0.000216, pi_batch=0.000216, match=True
  Year 4: pi_full=0.000253, pi_batch=0.000253, match=True
  Year 5: pi_full=0.000280, pi_batch=0.000280, match=T